In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib 
%run functions.py

Using matplotlib backend: TkAgg


In [2]:
def show(img):
    cv2.imshow('img',img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def draw(row,img):
    img_copy = img.copy()
    cv2.line(img_copy,(0,row),(img_copy.shape[1]-1,row),(255,0,0),1)
    show(img_copy)
    
def draw_lines(img):
    img_ = img.copy()
    h,w = img.shape
    p = 8;
    rate = int(h/p)
    answer = []
    for r in range(p):
        cv2.line(img_,(0,r*rate),(img_.shape[1]-1,r*rate),(255,0,0),1)
        answer.append(r*rate)
    show(img_)
    return answer
    
def process_(lista):
    result = []

    for i in range(len(lista)):
        val = 0
        '''
        if i == len(lista)-4:
            val = (lista[i]+lista[i+1]+lista[i+2]+lista[i+3])/4
        elif i == len(lista)-3:
            val = (lista[i]+lista[i+1]+lista[i+2])/3
        '''
        if i == len(lista)-2:
            val = (int(lista[i])+int(lista[i+1]))/2
        elif i == len(lista)-1:
            val = int(lista[i])
        else:
            val = (int(lista[i])+int(lista[i+1])+int(lista[i+2]))/3
            #if i == 0:
             #   print(lista[i]+lista[i+1]+lista[i+2])
        result.append(val)

    return result


In [3]:
img = cv2.imread('anillos.png')
img_gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
img_bin = cv2.threshold(img_gray, 100, 255, cv2.THRESH_BINARY)[1]

In [4]:
cap = cv2.VideoCapture('PadronAnillos_01.avi')
 
if (cap.isOpened()== False): 
    print("Error opening video stream or file")

In [5]:
show(img_bin)

In [6]:
y = img_gray[92]
x = [i for i in range(y.shape[0])]

In [7]:
plt.plot(x,y,'ro',x,y,'k')

In [8]:
y2 =img_gray[337]
plt.plot(x,y2,'ro',x,y2,'k')

In [7]:
ret, frame = cap.read()

In [8]:
frame_gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
thresh = 100
frame_bin = cv2.threshold(frame_gray, thresh, 255, cv2.THRESH_BINARY)[1]

In [9]:
#draw(220, frame_gray)
#%matplotlib inline
rows = draw_lines(frame_gray)
x = [i for i in range(frame_gray.shape[1])]

y = frame_gray[rows[1]]
print rows
plt.plot(x,y,'ro',x,y,'k')


[0, 90, 180, 270, 360, 450, 540, 630]


In [11]:
y = frame_gray[rows[4]]
plt.plot(x,y,'ro',x,y,'k')

In [28]:
#%matplotlib 
y = frame_bin[rows[5]]
plt.plot(x,y,'ro',x,y,'k')

In [37]:
%matplotlib
y_ = y.tolist()

media = process_(y_)
media5 = process5(y_)
final = [abs(y_[i]-media[i]) for i in range(len(y_))]
final5 = [abs(y_[i]-media5[i]) for i in range(len(y_))]

plt.plot(x,frame_bin[rows[3]],'k')
plt.plot(x,final,'b--')
plt.plot(x,final5,'ro',x,final5,'r--')

Using matplotlib backend: TkAgg


In [12]:
# ya tenemos la señal ahora encontrar las frecuencias
def frecuencias(signal):
    points = []
    for i in range(len(signal)-1):
        s,ns = signal[i], signal[i+1]
        if s == 255 and ns == 0:
            points.append(i)
    
    answer = []
    for i in range(len(points)-1):
        answer.append(points[i+1]-points[i])
    return answer, points

# comprobemos que ondas se parecen
def match_signals(fcs):
    fcs.append(0)
    match = 0
    answer = []
    for i in range(len(fcs)-1):
        f1, f2 = fcs[i], fcs[i+1]
        fm = (f1+f2)/2
        if abs(f1-fm) < 0.2*f1:
            match += 1
        else:
            # comprobar si en el pasado hay una secuencia se ondas parecidas
            # asumimos por lo menos 3 patrones reconocidos
            if match > 2:
                answer.append((i-match, i))
            match = 0
    return answer

# encontrar flanco de bajada blanco->negro
def find_edge(lista):
    for i in range(len(lista)-1):
        a, n = lista[i], lista[i+1]
        if a == 255 and n == 0:
            return i
    return -1
# encontrar que row contiene la columna con el primer flanco de bajada blanco->negro
def find_row_with_min(img, row, col):
    pad = 3 # vamos a comenzar tres px antes
    min_r = row
    flag = 0
    # buscamos abajo maximo 10 rows
    for i in range(50):
        px1, px2 = img[min_r][col], img[min_r][col+1]
        px1_, px2_ = img[min_r+1][col], img[min_r+1][col+1]
        d1 = abs(int(px1)-int(px1_))
        d2 = abs(int(px2_)-int(px2))

        if d2 == 255:
            flag += 1
            break
        if d1 == 0: # significa que para esos 2 pxs las dos rows son iguales
            min_r += 1
        else:       # tenemos que seguir bajando
            flag += 1
            min_r += 1
            # encontrar el px blanco
            while(img[min_r][col] == 0):
                col -= 1
            
    # si no encontro abajo entonces buscamos arriba
    # flag==1:tenemos que subir , flag>1 ya se encontro el la row 
    if flag == 1:
        min_r = row
        for i in range(50):
            px1, px2 = img[min_r][col], img[min_r][col+1]
            px1_, px2_ = img[min_r-1][col], img[min_r-1][col+1]
            d1 = abs(int(px1)-int(px1_))
            d2 = abs(int(px2)-int(px2_))
            if d2 == 255:
                break
            if d1 == 0: # significa que para esos 2 pxs las dos rows son iguales
                min_r -= 1
            else:       # tenemos que seguir bajando
                min_r -= 1
                # encontrar el px blanco
                while(img[min_r][col] == 0):
                    col -= 1
     
    return min_r, col
    
def new_col_in_new_frame(img, row, col):
    px = img[row][col]
    if px == 0: # caso 1
        while(img[row][col] == 0):
            col -= 1
    else: # caso 2
        while(img[row][col+1]==255):
            col += 1
    return col

In [14]:
sample = frame_bin[rows[3]]
#sample = img_bin[271]
fcs, points = frecuencias(sample)
m_s = match_signals(fcs)
index = m_s[0][0]
col = points[index]
r, c = find_row_with_min(frame_bin, rows[3], col)

print r,c
print

#new_c = new_col_in_new_frame(frame2_bin, r, c)
#r, c = find_row_with_min(frame2_bin, r, new_c)
#print r,c

285 376



In [2]:
cap = cv2.VideoCapture('calibration_mslifecam.avi')
row = 240#271
col = 0
flag = True
i = 0
cv2.namedWindow('image',0)
cv2.namedWindow('sample',0)
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    if ret== False: break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    binary = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY)[1]
    '''
    if flag:
        binary = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY)[1]
    else:
        sub_img = gray[row:row+600, col:col+440]
        ret, img_b = cv2.threshold(sub_img, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        binary = cv2.threshold(gray, ret, 255, cv2.THRESH_BINARY)[1]
        cv2.imshow('sample',sub_img)
    
    # processing first time
    if flag:
        antigua = gray
        fcs, points = frecuencias(binary[row])
        m_s = match_signals(fcs)
        index = m_s[0][0]
        col = points[index]
        row, col = find_row_with_min(binary, row, col)
        flag = False
    
    else:
        new_c = new_col_in_new_frame(binary, row, col)
        row, col = find_row_with_min(binary, row, new_c)
    
    cv2.line(gray,(0,row),(gray.shape[1]-1,row),(255,0,0),1)
    cv2.line(gray,(col,0),(col,gray.shape[0]-1),(255,0,0),1)
    '''
    
    cv2.imshow('image', binary)
    
    if cv2.waitKey(100) & 0xFF == ord('q'):
        break
    
    #if (i > 220 and i< 231):
     #   cv2.imwrite(str(i)+'.jpg', gray)
    print i, row, col
    antigua = gray
    cv2.imwrite(str(i)+'.jpg', frame)
    i += 1
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

0 240 0
1 240 0
2 240 0
3 240 0
4 240 0
5 240 0
6 240 0
7 240 0
8 240 0
9 240 0
10 240 0
11 240 0
12 240 0
13 240 0
14 240 0
15 240 0
16 240 0
17 240 0
18 240 0
19 240 0
20 240 0
21 240 0
22 240 0
23 240 0
24 240 0
25 240 0
26 240 0
27 240 0
28 240 0
29 240 0
30 240 0
31 240 0
32 240 0
33 240 0
34 240 0
35 240 0
36 240 0
37 240 0
38 240 0
39 240 0
40 240 0
41 240 0
42 240 0
43 240 0
44 240 0
45 240 0
46 240 0
47 240 0
48 240 0
49 240 0
50 240 0
51 240 0
52 240 0
53 240 0
54 240 0
55 240 0
56 240 0
57 240 0
58 240 0
59 240 0
60 240 0
61 240 0
62 240 0
63 240 0
64 240 0
65 240 0
66 240 0
67 240 0
68 240 0
69 240 0
70 240 0
71 240 0
72 240 0
73 240 0
74 240 0
75 240 0
76 240 0
77 240 0
78 240 0
79 240 0
80 240 0
81 240 0
82 240 0
83 240 0
84 240 0
85 240 0
86 240 0
87 240 0
88 240 0
89 240 0
90 240 0
91 240 0
92 240 0
93 240 0
94 240 0
95 240 0
96 240 0
97 240 0
98 240 0
99 240 0
100 240 0
101 240 0
102 240 0
103 240 0
104 240 0
105 240 0
106 240 0
107 240 0
108 240 0
109 240 0
110 240 0


In [11]:
img = cv2.imread('152.jpg',0)
ret, img_b = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
print ret
res = np.hstack((img,img_b))
show(img_b)

168.0


In [50]:
img1 = cv2.imread('56.jpg',0)
img2 = cv2.imread('57.jpg',0)
img3 = cv2.imread('58.jpg',0)
show(img3-img2)

In [9]:
cap = cv2.VideoCapture('Kinect2_rgb.avi')
ret, frame2 = cap.read()
draw(240,frame2[0:600])
cap.release()

In [52]:
#show(frame_bin[0:600])
#draw(rows[3], frame_gray)

frame2_gray = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
frame2_bin = cv2.threshold(frame2_gray, thresh, 255, cv2.THRESH_BINARY)[1]

cv2.imshow('img1',frame_bin[0:600])
cv2.imshow('img2', frame2_bin[0:600])
cv2.waitKey(0)
cv2.destroyAllWindows()


In [43]:
x = [i for i in range(img_bin.shape[1])]
plt.plot(x,img_bin[204],'k')

In [19]:
# veamos en el espacio de la frecuencia
#img = cv2.imread('anillos.png',0)
img = frame_gray
img_float32 = np.float32(img)

dft = cv2.dft(img_float32, flags = cv2.DFT_COMPLEX_OUTPUT)
dft_shift = np.fft.fftshift(dft)

magnitude_spectrum = 20*np.log(cv2.magnitude(dft_shift[:,:,0],dft_shift[:,:,1]))

plt.imshow(magnitude_spectrum, cmap = 'gray')
#cv2.namedWindow('fft',0)
#cv2.imshow()
#plt.subplot(121),plt.imshow(img, cmap = 'gray')
#plt.title('Input Image'), plt.xticks([]), plt.yticks([])
#plt.subplot(122),plt.imshow(magnitude_spectrum, cmap = 'gray')
#plt.title('Magnitude Spectrum'), plt.xticks([]), plt.yticks([])
#plt.show()

In [ ]:
y = frame_gray[220]
x = [i for i in range(y.shape[0])]
plt.plot(x,y,'ro',x,y,'k')

In [26]:
#plt.plot(x,y,'ro',x,y,'k')

In [51]:
b = ([2 ,3],[4,5])

[2, 3]